In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import warnings 

from IPython.display import display, HTML

warnings.filterwarnings("ignore")

plt.rcParams["patch.force_edgecolor"] = True

## 1. Структура данных

In [2]:
data = pd.read_csv(
    "./data/data.csv", 
    encoding="ISO-8859-1", 
    dtype={'CustomerID': str,'InvoiceID': str}
)
print('Data shape: {}'.format(data.shape))
data.head(5)

Data shape: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [ ]:
#Анализ номеров счетов (InvoiceNo)
print('Уникальные префиксы в InvoiceNo:')
invoice_prefixes = data['InvoiceNo'].str.extract(r'^(\D+)', expand=False).dropna().unique()
print(f'Найдены префиксы: {invoice_prefixes}')
#Разделение на обычные транзакции и возвраты
data['Is_Cancellation'] = data['InvoiceNo'].str.startswith('C')
data['Is_Discount'] = data['StockCode'].str.contains('D|POST|M', case=False, na=False)
print(f'\nКоличество транзакций с возвратами: {data["Is_Cancellation"].sum()}')
print(f'Количество транзакций со скидками: {data["Is_Discount"].sum()}')

Уникальные префиксы в InvoiceNo:
Найдены префиксы: ['C' 'A']

Количество транзакций с возвратами: 9288
Количество транзакций со скидками: 6957
